# MPIEvaluator: Run on multi-node HPC systems

The `MPIEvaluator` is a new addition to the EMAworkbench that allows experiment execution on multi-node systems, including high-performance computers (HPCs). This capability is particularly useful if you want to conduct large-scale experiments that require distributed processing. Under the hood, the evaluator leverages the `MPIPoolExecutor` from [`mpi4py.futures`](https://mpi4py.readthedocs.io/en/stable/mpi4py.futures.html).

#### Limiations
- Currently, the MPIEvaluator is only tested on Linux, while it might work on other operating systems.
- Currently, the MPIEvaluator only works with Python-based models, and won't work with file-based model types (like NetLogo or Vensim).
- The MPIEvaluator is most useful for large-scale experiments, where the time spent on distributing the experiments over the cluster is negligible compared to the time spent on running the experiments. For smaller experiments, the overhead of distributing the experiments over the cluster might be significant, and it might be more efficient to run the experiments locally.

The MPIEvaluator is experimental and its interface and functionality might change in future releases. We appreciate feedback on the MPIEvaluator, share any thoughts about it at https://github.com/quaquel/EMAworkbench/discussions/311.

#### Contents
This tutorial will first show how to set up the environment, and then how to use the MPIEvaluator to run a model on a cluster. Finally, we'll use the [DelftBlue supercomputer](https://doc.dhpc.tudelft.nl/delftblue/) as an example, to show how to run on a system which uses a SLURM scheduler.

## 1. Setting up the environment
To use the MPIEvaluator, MPI and mpi4py must be installed. 

Installing MPI on Linux typically involves the installation of a popular MPI implementation such as OpenMPI or MPICH. Below are the instructions for installing OpenMPI:

### 1a. Installing OpenMPI

_If you use conda, it might install MPI automatically along when installing mpi4py (see 1b)._

You can install OpenMPI using you package manager. First, update your package repositories, and then install OpenMPI:
   
   For **Debian/Ubuntu**:
   ```bash
   sudo apt update
   sudo apt install openmpi-bin libopenmpi-dev
   ```

   For **Fedora**:
   ```bash
   sudo dnf check-update
   sudo dnf install openmpi openmpi-devel
   ```

   For **CentOS/RHEL**:
   ```bash
   sudo yum update
   sudo yum install openmpi openmpi-devel
   ```

Many times, the necessary environment variables are automatically set up. You can check if this is the case by running the following command:

   ```bash
   mpiexec --version
   ```

If not, add OpenMPI's `bin` directory to your `PATH`:

   ```bash
   export PATH=$PATH:/usr/lib/openmpi/bin
   ```

### 1b. Installing mpi4py
The python package mpi4py needs to installed as well. This is most easily done [from PyPI](https://pypi.org/project/mpi4py/), by running the following command:
```bash
pip install -U mpi4py
```

## 2. Creating a model
First, let's set up a minimal model to test with. This can be any Python-based model, we're using the [`example_python.py`](https://emaworkbench.readthedocs.io/en/latest/examples/example_python.html) model from the EMA Workbench documentation as example.

We recommend crafting and testing your model in a separate Python file, and then importing it into your main script. This way, you can test your model without having to run it through the MPIEvaluator, and you can easily switch between running it locally and on a cluster.

### 2a. Define the model
First, we define a Python model function.

In [ ]:
def some_model(x1=None, x2=None, x3=None):
    return {"y": x1 * x2 + x3}

Now, create the EMAworkbench model object, and specify the uncertainties and outcomes:

In [ ]:
from ema_workbench import Model, RealParameter, ScalarOutcome, ema_logging, perform_experiments

if __name__ == "__main__":
    # We recommend setting pass_root_logger_level=True when running on a cluster, to ensure consistent log levels.
    ema_logging.log_to_stderr(level=ema_logging.INFO, pass_root_logger_level=True)

    ema_model = Model("simpleModel", function=some_model)  # instantiate the model

    # specify uncertainties
    ema_model.uncertainties = [
        RealParameter("x1", 0.1, 10),
        RealParameter("x2", -0.01, 0.01),
        RealParameter("x3", -0.01, 0.01),
    ]
    # specify outcomes
    ema_model.outcomes = [ScalarOutcome("y")]

### 2b. Test the model
Now, we can run the model locally to test it:

In [ ]:
from ema_workbench import SequentialEvaluator

with SequentialEvaluator(ema_model) as evaluator:
    results = perform_experiments(ema_model, 100, evaluator=evaluator)

In this stage, you can test your model and make sure it works as expected. You can also check if everything is included in the results and do initial validation on the model, before scaling up to a cluster.

## 3. Run the model on a MPI cluster
Now that we have a working model, we can run it on a cluster. To do this, we need to import the `MPIEvaluator` class from the `ema_workbench` package, and instantiate it with our model. Then, we can use the `perform_experiments` function as usual, and the MPIEvaluator will take care of distributing the experiments over the cluster. Finally, we can save the results to a pickle file, as usual.

In [ ]:
# ema_example_model.py
from ema_workbench import (
    Model,
    RealParameter,
    ScalarOutcome,
    ema_logging,
    perform_experiments,
    MPIEvaluator,
    save_results,
)
import pickle

def some_model(x1=None, x2=None, x3=None):
    return {"y": x1 * x2 + x3}

if __name__ == "__main__":
    ema_logging.log_to_stderr(level=ema_logging.INFO, pass_root_logger_level=True)

    ema_model = Model("simpleModel", function=some_model)

    ema_model.uncertainties = [
        RealParameter("x1", 0.1, 10),
        RealParameter("x2", -0.01, 0.01),
        RealParameter("x3", -0.01, 0.01),
    ]
    ema_model.outcomes = [ScalarOutcome("y")]

    # Note that we switch to the MPIEvaluator here
    with MPIEvaluator(ema_model) as evaluator:
        results = evaluator.perform_experiments(scenarios=10000)

    # Save the results
    save_results(results, "ema_mpi_test.tar.gz")

To run this script on a cluster, we need to use the `mpiexec` command:
   ```bash
   mpiexec python3 -m mpi4py.futures ema_example_model.py
   ```
This command will execute the `ema_example_model.py` Python script using MPI, leveraging the `mpi4py.futures` module for parallel processing. The number of processes and other MPI-specific configurations would be inferred from default settings or any configurations provided elsewhere, such as in an MPI configuration file or additional flags to `mpiexec` (not shown in the provided command).

The output of the script will be saved to the `ema_mpi_test.pickle` file, which can be loaded and analyzed as usual.